In [1]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-04 00:12:00--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-04 00:12:00--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca86e58542a85b376782ca64c7e.dl.dropboxusercontent.com/cd/0/inline/BN0cnnlkEylApFl-AM1WqaGrcMAt_0rAXz7fmAyM6jGKJqF83U9nUtc79a90o2ANneR1oQzqDhxUrGLJ_qXnFZ-tIKgOk1X0ybbDVJWnEZYaAFebxEzWCJdQrKBy4tvRPI-l1y0YrTR7x-wG0G34XAwi/file# [following]
--2021-05-04 00:12:01--  https://uca86e58542a85b376782ca64c7e.dl.dropboxusercontent.com/cd/0/inline/BN0cnnlkEylApFl-AM1WqaGrcMAt_0rAXz7fmAyM6jGKJqF83U9nUtc79a90o

In [2]:
!nvidia-smi

Tue May  4 00:13:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pickle

import torch
!pip install transformers
from transformers import BertForNextSentencePrediction, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
from scipy import stats
import sklearn
import collections
import random

from google.colab import drive

random.seed(2021)
drive.mount('/content/gdrive')

     |████████████████████████████████| 2.1MB 9.9MB/s 
     |████████████████████████████████| 901kB 63.5MB/s 
     |████████████████████████████████| 3.3MB 60.7MB/s 
Mounted at /content/gdrive


In [4]:
# load sample data
# format { post_id: [post, [comment1, comment2, ... ] }

data_file_path = 'grouped_data.pickle'

def get_best_comm(comms):
  best_comm = None
  best_score = -np.inf

  for comm in comms:
    if comm['score'] > best_score:
      best_comm = comm
      best_score = comm['score']

    return best_comm

def load_data(file_path):
    posts_comments = []
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        for post_id, val in tqdm(list(data.items())[:5000]):
            post = val[0]
            comms = val[1]
            # best_comm = get_best_comm(comms)

            # if len(post['title']) < 500 and len(best_comm['body']) < 500 and best_comm['body'] != '[deleted]' and best_comm['body'] != '[removed]':
            #   posts_comments.append((post['title'], best_comm['body']))

            for com in comms:
                try:
                    if len(post['title']) < 500 and len(com['body']) < 500 and com['body'] != '[deleted]' and com['body'] != '[removed]':
                        posts_comments.append((post['title'], com['body']))
                except:
                    pass
                
    return posts_comments
    
posts_comms = load_data(data_file_path)
len(posts_comms)

65242

In [5]:
post_comms_realfake = []
labels = []
for post, comment in posts_comms:
  while True:
    rpost, rcomment = random.choice(posts_comms)
    if post != rpost:
      break
  
  post_comms_realfake.append((post, comment))
  labels.append(0) # comment \in post
  post_comms_realfake.append((post, rcomment))
  labels.append(1) # comment \nin post

len(post_comms_realfake)

130484

In [6]:
# split text
train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.25, shuffle=True)
# train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.01, train_size=.04, shuffle=True)

In [7]:
# load tokenizer
tokenizer = BertTokenizerFast.from_pretrained('distilbert-base-cased', use_cache=True)

In [8]:
# tokenize train data

train_post = []
train_comm = []

for x in tqdm(train_texts):
    train_post.append(x[0])
    train_comm.append(x[1])

train_texts = None
train_encodings = tokenizer(text=train_post, text_pair=train_comm, truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
# tokenize test data

test_post = []
test_comm = []

for x in tqdm(test_texts):
    test_post.append(x[0])
    test_comm.append(x[1])

test_texts = None

test_encodings = tokenizer(text=test_post, text_pair=test_comm, truncation=True, padding=True)


In [10]:
# load model
model = BertForNextSentencePrediction.from_pretrained('distilbert-base-cased', num_labels=2)
model.train()

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing BertForNextSentencePrediction: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ff

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [11]:
# create dataset class and load encodings and associated labels to it

class upvote_prediction_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = upvote_prediction_dataset(train_encodings, train_labels)
test_dataset = upvote_prediction_dataset(test_encodings, test_labels)

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    logging_dir='./logs',            # directory for storing logs
    logging_steps = 10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()


Step,Training Loss
10,0.713300
20,0.716200
30,0.682600
40,0.735500
50,0.746000
60,0.725800
70,0.729900
80,0.807800
90,0.711100
100,0.697600


TrainOutput(global_step=30585, training_loss=0.6953737976934193, metrics={'train_runtime': 11348.3368, 'train_samples_per_second': 2.695, 'total_flos': 9.635144263785272e+16, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 2471079936, 'init_mem_gpu_alloc_delta': 433776128, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -397328384, 'train_mem_gpu_alloc_delta': 1313640960, 'train_mem_cpu_peaked_delta': 416845824, 'train_mem_gpu_peaked_delta': 5908862464})

In [20]:
trainer.save_model('/content/gdrive/My Drive/relevance_model_nsp')

In [21]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [22]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [23]:
# load model
model = BertForNextSentencePrediction.from_pretrained('/content/gdrive/My Drive/relevance_model_nsp', num_labels=2)
model.eval()

model.to(device)
items = {key: torch.tensor(val).to(device) for key, val in train_encoding.items()}

In [27]:
preds = []
for batch_idxs in tqdm(batch(range(len(items['input_ids'])), 16)):
  batch_input_ids = items['input_ids'][batch_idxs]
  batch_attention_mask = items['attention_mask'][batch_idxs]
  batch_token_type_ids = items['token_type_ids'][batch_idxs]

  outs = model.forward(input_ids=batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids)
  probs = torch.softmax(outs.logits, 1)
  classes = torch.argmax(probs, dim=1)

  preds += list(classes.cpu().numpy())

In [28]:
sklearn.metrics.f1_score(preds, train_labels, average='weighted')

ValueError: ignored

In [ ]:
sklearn.metrics.confusion_matrix(preds, train_labels)